In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
import numpy as np
from sklearn.svm import SVC, NuSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

In [2]:
t_train = torch.load('../data/embeds/polyhope_train_english_bbu_multiclass.pt')
t_val = torch.load('../data/embeds/polyhope_val_english_bbu_multiclass.pt')
t_test = torch.load('../data/embeds/polyhope_test_english_bbu_multiclass.pt')

t5_train = torch.load('../data/embeds/polyhope_train_english_embeds_t5_large.pt')
t5_test = torch.load('../data/embeds/polyhope_test_english_embeds_t5_large.pt')
t5_val = torch.load('../data/embeds/polyhope_val_english_embeds_t5_large.pt')

s_train = torch.load('../data/embeds/polyhope_train_english_sentiment.pt')
s_val = torch.load('../data/embeds/polyhope_val_english_sentiment.pt')
s_test = torch.load('../data/embeds/polyhope_test_english_sentiment.pt')

In [3]:
polyhope_multi_labels = {'Not Hope':0, 'Generalized Hope':1, 'Realistic Hope':2, 'Unrealistic Hope':3}
polyhope_inv_multi_labels = {v: k for k, v in polyhope_multi_labels.items()}
polyhope_binary_labels = {'Hope':1, 'Not Hope':0}
polyhope_inv_binary_labels = {v: k for k, v in polyhope_binary_labels.items()}

In [4]:
df_train = pd.read_csv('../data/train/train_polyhope_english.csv')
train_bin = df_train['binary'].tolist()
train_multi = df_train['multiclass'].tolist()

df_val = pd.read_csv('../data/val/val_polyhope_english.csv')
val_bin = df_val['binary'].tolist()
val_multi = df_val['multiclass'].tolist()

df_test = pd.read_csv('../data/test/test_polyhope_english.csv')
test_bin = df_test['binary'].tolist()
test_multi = df_test['multiclass'].tolist()

In [5]:
t_train = np.array([x[0].numpy() for x in t_train])
t_val = np.array([x[0].numpy() for x in t_val])
t_test = np.array([x[0].numpy() for x in t_test])

s_train = np.array([x[0].numpy() for x in s_train])
s_val = np.array([x[0].numpy() for x in s_val])
s_test = np.array([x[0].numpy() for x in s_test])

In [7]:
t5_train = t5_train.numpy()
t5_val = t5_val.numpy()
t5_test = t5_test.numpy()

In [26]:
X = np.concatenate((t_train, s_train), axis=1)
Y = [polyhope_multi_labels[i] for i in train_multi]
# Y = [polyhope_binary_labels[i] for i in train_bin]

clf = SVC()
# clf.fit(X, Y)
clf.fit(t_train, Y)

# clf = make_pipeline(StandardScaler(), NuSVC())
# clf.fit(t_train, Y)

SVC()

In [ ]:
clf.score(t_test, [polyhope_multi_labels[i] for i in test_multi])
# clf.score(t_test, [polyhope_binary_labels[i] for i in test_bin])

In [25]:
clf.score(t_test, [polyhope_multi_labels[i] for i in test_multi])

0.6715116279069767

In [9]:
clf.score(t_train, [polyhope_multi_labels[i] for i in train_multi])

0.7688953488372093

In [22]:
clf.score(np.concatenate((t_test, s_test), axis=1), [polyhope_multi_labels[i] for i in test_multi])

0.6656976744186046

In [12]:
clf.score(np.concatenate((t_test, s_test), axis=1), [polyhope_multi_labels[i] for i in test_multi])

0.7645348837209303

In [19]:
pred = []
val_bin_c = [polyhope_multi_labels[i] for i in val_multi]
for i, text in enumerate(tqdm(val_texts)):
    pred.append(clf.predict([np.concatenate((t_val[i], s_val[i]), axis=None)]))

print(accuracy_score(val_bin_c, pred))
print(classification_report(val_bin_c, pred))

100%|██████████████████████████████████████████████████████████| 1032/1032 [00:01<00:00, 1015.49it/s]

0.7858527131782945
              precision    recall  f1-score   support

           1       0.86      0.87      0.87       502
           2       0.76      0.74      0.75       300
           3       0.65      0.68      0.66       128
           4       0.64      0.62      0.63       102

    accuracy                           0.79      1032
   macro avg       0.73      0.73      0.73      1032
weighted avg       0.79      0.79      0.79      1032



In [8]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
        random_state=0).fit(X, Y)
# gpc.predict_proba(X[:2,:])

NameError: name 'y' is not defined

In [9]:
x_test = t_val.numpy()
y_test = Y = [polyhope_binary_labels[i] for i in val_bin]

In [10]:
gpc.score(x_test, y_test)

0.8032945736434108

In [3]:
txt = '/asdf/adfa/sdfadf/adfadff/dsd/test_polyhope_english_cleaned_noemoji.csv'
tok = txt.split('.')[0].split('_')

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader, Dataset
import torch
import pandas as pd
from tqdm import tqdm

class TextDataset(Dataset):
	def __init__(self, texts, tokenizer, max_length):
		self.texts = texts
		self.tokenizer = tokenizer
		self.max_length = max_length
        
	def __len__(self):
		return len(self.texts)
        
	def __getitem__(self, idx):
		text = f'translate Spanish to English: {self.texts[idx]}'
		encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
		return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten()}

device = torch.device('cuda')
model_name = 'google/flan-t5-small'

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

df = pd.read_csv('../data/train/train_polyhope_spanish.csv')
texts = df['text'].tolist()
tds = TextDataset(texts, tokenizer, 128)
tdd = DataLoader(tds, batch_size=64)

tran_texts = []

for batch in tqdm(tdd):
	input_ids = batch['input_ids'].to(device)
	output = model.generate(input_ids, max_length=300)
	for i in output:
		tran_texts.append(tokenizer.decode(i, skip_special_tokens=True))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|█████████████████████████████████████████| 108/108 [21:27<00:00, 11.92s/it]


In [2]:
for i, j in zip(texts[:5], tran_texts[:5]):
    print(f'{i} : {j}')

Mientras me persigno y le rezo a la Virgen del Carmen. Así e'. #URL# : 
No.. Yo ya no estoy para esperar.. Ni para rogar, ni competir con nadie.. Que va... Quien me da recibe.. #URL# : I don't want to wait for you to wait.. I don't want to wait, I want to buy with you.. I want to get you a refund..
Creo que estamos ante el mejor episodio de esta primera temporada. #TheLastOfUs : I think we are at the best episode of this first week. #TheLastOfUs
Bueno ojalá q cuando llegue este fin de semana pueda realmente quedarme en mi camaaaa sin movermeeee hasta que me pudra. Fin. : Good luck when you come to this week's end can really really want to do in my way without a move until I can get it.
#USER# #USER# Marcos de boludo no tiene un pelo pero no creo que sepa con certeza del acomodo de ellas, máximo y sospecha, afuera va saber y va entender muchas cosas que seguro no le cierran, y se va enterar lo mal que hablaron ellas de él, Agus le va mostrar los clips. : #USER#USER#USER# Marcos of bolud

In [13]:
input_text = ["translate English to Spanish: How old are you?", "translate English to German: How old are you?"]
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=500)
for i in outputs:
    print(tokenizer.decode(i, skip_special_tokens=True))

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [9]:
outputs

tensor([[    0,  2739,     3,   362,     3,    49,     3, 26040,    58,     1],
        [    0,  2739,     3,   362,     3,    49,     3, 26040,    58,     1]],
       device='cuda:0')